In [4]:
import pandas as pd
import os
import ast
 
# List of file names
bath = "Dataset\\"
file_names = ['bangalore_cars.xlsx', 'chennai_cars.xlsx', 'delhi_cars.xlsx',
              'hyderabad_cars.xlsx', 'jaipur_cars.xlsx', 'kolkata_cars.xlsx']
dfs = []
 
for file in file_names:
    df = pd.read_excel(f"{bath}{file}")
   
    state = os.path.splitext(file)[0].split('_')[0]
   
    df['state'] = state
   
    dfs.append(df)
 
merged_df = pd.concat(dfs, ignore_index=True)
 
# merged_df.to_excel('merged_cars_data.xlsx', index=False)
 
print("Files merged successfully. Output saved as 'merged_cars_data.xlsx'")

Files merged successfully. Output saved as 'merged_cars_data.xlsx'


In [5]:
car_columns = ['new_car_detail',"new_car_overview","new_car_feature", "new_car_specs"]
 
def first_columns(car_data):
    import pandas as pd
    import ast
 
    # Function to convert stringified dictionaries into actual dictionaries
    def convert_to_dict(row):
        try:
            return ast.literal_eval(row)
        except ValueError:
            return None
 
    # Convert the stringified dictionaries to actual dictionaries
    data_as_dicts = [convert_to_dict(row) for row in car_data]
 
    # Function to flatten nested dictionaries and lists
    # {a:[b:{c:[d:f]]}
    def flatten_dict(d, parent_key='', sep='_'):
        items = []
        if isinstance(d, dict):
            for k, v in d.items():
                new_key = f"{parent_key}{k}" if parent_key else k
                if isinstance(v, dict):
                    items.extend(flatten_dict(v, new_key, sep=sep).items())
                elif isinstance(v, list):
                    for i, item in enumerate(v):
                        items.extend(flatten_dict(item, f"{new_key}_{i}", sep=sep).items())
                else:
                    items.append((new_key, v))
        elif isinstance(d, list):
            for i, item in enumerate(d):
                items.extend(flatten_dict(item, f"{parent_key}_{i}", sep=sep).items())
        else:
            items.append((parent_key, d))
        return dict(items)
 
    # Flatten each dictionary in the list
    flattened_data = [flatten_dict(d) for d in data_as_dicts if d is not None]
 
    # Create a DataFrame from the flattened data
    df = pd.DataFrame(flattened_data)
    return df
 
def four_column(car_data):
 
    # Function to convert stringified dictionaries into actual dictionaries
    def convert_to_dict(row):
        try:
            return ast.literal_eval(row)
        except ValueError:
            return None
 
    # Convert the stringified dictionaries to actual dictionaries
    data_as_dicts = [convert_to_dict(row) for row in car_data]
 
    # Extract key-value pairs from each dictionary into a flat structure
    def extract_key_value_pairs(d):
        flat_dict = {}
        try:
            if 'top' in d:
                for item in d['top']:
                    key = item['key']  # Extracting the key
                    value = item['value']  # Extracting the value
                    flat_dict[key] = value
        except:
            key = ''
            value = ''
            flat_dict[key] = value

        try:
            if 'data' in d:
                for section in d['data']:
                    if 'list' in section:
                        for item in section['list']:
                            key = item['key']  # Extracting the key
                            value = item['value']  # Extracting the value
                            flat_dict[key] = value
        except:
            key = ''
            value = ''
            flat_dict[key] = value        
       
        return flat_dict
 
    # Apply extraction to each dictionary and create a flattened list of rows
    flattened_data = [extract_key_value_pairs(d) for d in data_as_dicts if d is not None]
 
    # Create a DataFrame from the flattened data
    df = pd.DataFrame(flattened_data)
    return df

In [6]:
import pandas as pd
 
# Initialize an empty DataFrame to store the concatenated results
final_df = pd.DataFrame()
 
# List of car columns (assumed to be correct)
car_columns = ['new_car_detail', 'new_car_overview', 'new_car_feature', 'new_car_specs']
 
# Assuming `merged_df` is the merged DataFrame that you have already created
for i in range(len(car_columns)):
    if i == 0:
        w = first_columns(merged_df[car_columns[i]])  # Flatten first columns
    elif i>0:
        w = four_column(merged_df[car_columns[i]])  # Flatten complex columns
 
    # Concatenate the current DataFrame 'w' with 'final_df'
    final_df = pd.concat([final_df, w], axis=1, ignore_index=False)  # axis=1 concatenates columns
final_df['city']=merged_df['state']
# Now, `final_df` will contain the flattened data from all iterations
print(final_df)

      it      ft         bt        km transmission  ownerNo      owner  \
0      0  Petrol  Hatchback  1,20,000       Manual        3  3rd Owner   
1      0  Petrol        SUV    32,706       Manual        2  2nd Owner   
2      0  Petrol  Hatchback    11,949       Manual        1  1st Owner   
3      0  Petrol      Sedan    17,794       Manual        1  1st Owner   
4      0  Diesel        SUV    60,000       Manual        1  1st Owner   
...   ..     ...        ...       ...          ...      ...        ...   
8364   0  Petrol  Hatchback    10,000       Manual        1  1st Owner   
8365   0  Petrol  Hatchback  1,20,000       Manual        1  1st Owner   
8366   0  Petrol      Sedan    50,000    Automatic        3  3rd Owner   
8367   0  Petrol  Hatchback    40,000       Manual        1  1st Owner   
8368   0  Diesel        SUV  1,20,000       Manual        2  2nd Owner   

                oem                  model  modelYear  ...  Rear Brake Type  \
0            Maruti         Maru

In [7]:
pd.set_option('display.max_columns', None)
df = final_df

In [9]:
def clean_km(km):
    km = int(km.replace(',','').strip())
    return km
df['km']=df['km'].apply(clean_km)

null_counts = df.isnull().sum()
# Displaying the entire column names with their null count
for column, count in null_counts.items():
    print(f"Column: {column}, Null Count: {count}")

df['ownerNo']=df['ownerNo'].astype(int)
df['centralVariantId']=df['centralVariantId'].astype(int)
df['modelYear']=df['modelYear'].astype(int)


def clean_price(price):
    price = price.replace(',','').replace('₹','').strip()
    try:
        if 'Lakh' in price:
            price = float(price.replace('Lakh','').strip()) * 100000
        elif 'Crore' in price:
            price = float(price.replace('Crore','').strip()) * 10000000
        else:
            price = float(price)
    except ValueError:
        return None
    
    return price

df['price']=df['price'].apply(clean_price)

df['price']=df['price'].astype(int)

def clean_ed(eng):
    if isinstance(eng, str):  # Check if eng is a string
        eng = int(eng.replace('CC', '').strip())  # Clean the string
        return eng
    return eng 

df['Engine']=df['Engine'].apply(clean_ed)

df = df.dropna(subset=['Engine'])

df['Engine']=df['Engine'].astype(int)




def clean_ml(Mileage):

    if isinstance(Mileage, str):  # Check if Mileage is a string
        Mileage = Mileage.replace('kmpl', '').replace('km/kg','').strip()
        Mileage = float(Mileage)
    return Mileage
df['Mileage']=df['Mileage'].apply(clean_ml)

df = df.dropna(subset=['Seating Capacity'])
df = df.drop(columns=["Seats"])

Column: it, Null Count: 0
Column: ft, Null Count: 0
Column: bt, Null Count: 0
Column: km, Null Count: 0
Column: transmission, Null Count: 0
Column: ownerNo, Null Count: 0
Column: owner, Null Count: 0
Column: oem, Null Count: 0
Column: model, Null Count: 0
Column: modelYear, Null Count: 0
Column: centralVariantId, Null Count: 0
Column: variantName, Null Count: 0
Column: price, Null Count: 0
Column: priceActual, Null Count: 0
Column: priceSaving, Null Count: 0
Column: priceFixedText, Null Count: 8369
Column: trendingTextimgUrl, Null Count: 0
Column: trendingTextheading, Null Count: 0
Column: trendingTextdesc, Null Count: 0
Column: Registration Year, Null Count: 52
Column: Insurance Validity, Null Count: 4
Column: Fuel Type, Null Count: 0
Column: Seats, Null Count: 6
Column: Kms Driven, Null Count: 2
Column: RTO, Null Count: 888
Column: Ownership, Null Count: 32
Column: Engine Displacement, Null Count: 4
Column: Transmission, Null Count: 0
Column: Year of Manufacture, Null Count: 20
Colum

C:\Users\infog\AppData\Local\Temp\ipykernel_36092\388357363.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Engine']=df['Engine'].astype(int)
C:\Users\infog\AppData\Local\Temp\ipykernel_36092\388357363.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Mileage']=df['Mileage'].apply(clean_ml)


In [11]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
 
def co(column_data, price,column_):
    
    data = pd.DataFrame()
    data[column_] = column_data
    data['price'] = price
    f = data[column_].isnull().sum()
    data = data.dropna()
    le = LabelEncoder()
    
    # Check if the column is non-numerical
    if data[column_].dtype == 'object':
        data[column_] = le.fit_transform(data[column_].astype(str))
    
    correlation = data[i].corr(data['price'])
    return correlation, f
 
column_s = df.columns
# df_cleaned = df.drop(columns=["Seats"])

for i in column_s:
    t = co(df[i],df["price"],i)
    print(i,t)




 

c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\infog\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\function_bas

it (nan, 0)
ft (-0.18134117171709702, 0)
bt (0.26557820440500396, 0)
km (-0.07731156737902267, 0)
transmission (-0.4225535727873822, 0)
ownerNo (-0.120160374550546, 0)
owner (-0.120160374550546, 0)
oem (-0.034502979318195424, 0)
model (-0.00808243327869705, 0)
modelYear (0.29985527995117445, 0)
centralVariantId (0.1833353917257284, 0)
variantName (-0.03753379168837099, 0)
price (1.0, 0)
priceActual (-0.11022783003570773, 0)
priceSaving (nan, 0)
priceFixedText (nan, 8359)
trendingTextimgUrl (nan, 0)
trendingTextheading (nan, 0)
trendingTextdesc (nan, 0)
Registration Year (0.07078474484529845, 50)
Insurance Validity (-0.1408277761245357, 4)
Fuel Type (-0.18134117171709702, 0)
Kms Driven (-0.06923551810574322, 2)
RTO (-0.05773468089915506, 885)
Ownership (-0.1091261390276018, 32)
Engine Displacement (0.14688331842475852, 0)
Transmission (-0.4225535727873822, 0)
Year of Manufacture (0.30233066578774653, 18)
 (nan, 57)
Mileage (-0.33459405482007415, 283)
Engine (0.5366301966387229, 0)
Max P

In [12]:
data = pd.DataFrame()
data['ft'] = df['ft']
data['km'] = df['km']
data['transmission'] = df['transmission']
data['ownerNo'] = df['ownerNo']
data['oem'] = df['oem']
data['modelYear'] = df['modelYear']
data['price'] = df['price']
data['Insurance Validity'] = df['Insurance Validity']
data['Mileage'] = df['Mileage']
data['Engine'] = df['Engine']
data['city'] = df['city']
data['Seating Capacity'] = df['Seating Capacity']
data['Mileage'] = df['Mileage']
data['Gear Box'] = df['Gear Box']


# ft, km, transmission, owner no, Oem, ModelYear, price, insurance Validity, Mileage, engine, city, Seating Capacity

In [13]:
data.to_csv("data_1.csv",index=False)